 #### Simulation of Impulse Train Sampling
$x_p(t) = x(t)p(t)$

 p(t) : sampling function <br>
 T: sampling period <br> 
 $\omega _s = 2 \pi / T $ : sampling frequency = fundamental frequency of p(t)


 In the simulation, we use square function with different duty cycle than the unit impulse function



Sampling Theorem : condition for a sample rate for a discrete sample to capture all the information froma a continuous time signal of finite bandwidth
If a function have no frequency higher than B, the sampling rate should be higher 2B(nyquist rate)

When we sample, we multiply the signal in time domain with an impulse train of sampling frequency (fs). 
The impulse train in time domain gives an impulse train in frequency domain, which the impulse is fs apart.
And the input signal frequency response gets convoled with each impulse train, creating a shifted versions of the original signal frequency response.
When the signal is real, the signal is symmetric in frequency domain. So if the signal had a maximum frequency of B, the sampling frequency has to be twice the maximum frequency (2B) to avoide alising. 

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from commpy.filters import rcosfilter
from scipy import signal
from scipy.fft import fft, fftfreq

from ipywidgets import interact
import ipywidgets as widgets

In [46]:
# With square function and duty cycle

f = 50  # Frequency of the original signal
n = 1000*10**2   #number of points for the original signal
t = np.linspace(0,1,n)  #time range
x = 5*np.sin(2*np.pi*f*t)  #original signal
x_f = fft(x)    # original signal in frequency domain 
f = fftfreq(n, 1/n)[:n//2]  #frequency axis


fig = make_subplots(rows = 3, cols = 2, subplot_titles=("original", "original", "impulse train", "impulse train", "sampled signal", "sampled signal"))

fig.update_layout(title_text="sampling")

# time domain
fig.add_scatter(x = t[:n//2], y = x, row = 1, col = 1)
fig.add_scatter(x = t[:n//2], mode = "lines", row = 2, col = 1)
fig.add_scatter(x = t[:n//2], row = 3, col = 1)
fig.update_xaxes(title_text="time", row=3, col=1)

# #frequency domain
fig.add_scatter(x = f[:(f.size)//25], y = abs(x_f)[:n//2], row = 1, col = 2)
fig.add_scatter(x = f[:(f.size)//25], mode = "lines", row = 2, col = 2)
fig.add_scatter(x = f[:(f.size)//25], row = 3, col = 2)
fig.update_xaxes(title_text="freq", row=3, col=2)

fig.layout.update(showlegend=False)
figw = go.FigureWidget(fig)

@interact (fs = [48, 100, 200,500, 1000], duty = [0.001, 0.1, 0.3, 0.5, 0.7])

def update(fs = 200, duty = 0.001):
    p = 0.5*signal.square( t*2*np.pi*fs, 0.001) + 0.5
    s = x*p
    with figw.batch_update():
        figw.data[1].y = p
        figw.data[2].y = s
        figw.data[4].y = abs(fft(p))[:n//2]
        figw.data[5].y = abs(fft(s))[:n//2]
figw


interactive(children=(Dropdown(description='fs', index=2, options=(48, 100, 200, 500, 1000), value=200), Dropd…

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '7d51be2e-f1a2-43a0-9bef-c3c9f76b5acf',
 …

In [4]:
# With impulse train

f = 50  #Frequency of the input signal
n = 100000
t = np.linspace(0,1,n)
x = 5*np.sin(2*np.pi*f*t)  # input signal
x_f = fft(x)    # frequency domain of input signal
f = fftfreq(n, 1/n)

fig = make_subplots(rows = 3, cols = 2, subplot_titles=("original", "original", "impulse train", "impulse train", "sampled signal", "sampled signal"))

fig.update_layout(title_text="sampling")

# time domain
fig.add_scatter(x = t[:n//2], y = x, row = 1, col = 1)
fig.add_scatter(x = t[:n//2], mode = "lines", row = 2, col = 1)
fig.add_scatter(x = t[:n//2], row = 3, col = 1)
fig.update_xaxes(title_text="time", row=3, col=1)

# #frequency domain
fig.add_scatter(x = f, y = abs(x_f), row = 1, col = 2)
fig.add_scatter(x = f, mode = "lines", row = 2, col = 2)
fig.add_scatter(x = f, row = 3, col = 2)
fig.update_xaxes(title_text="freq", row=3, col=2)


fig.update_xaxes(range=[-100,100], row =1, col = 2)
fig.update_xaxes(range=[-1000,1000], row =2, col = 2)
fig.update_xaxes(range=[-500,500], row =3, col = 2)

fig.layout.update(showlegend=False)
figw = go.FigureWidget(fig)

@interact (fs = 200)#[48, 100, 200, 1000])

def update(fs = 200):
    A = np.arange(0,n,n//fs) # having fs samples from [0 to n]
    p = signal.unit_impulse(n, A) # Impulse train of frequency fs (sampling rate of fs)
    s = x*p  # sampled signal
    with figw.batch_update():
        figw.data[1].y = p
        figw.data[2].y = s
        figw.data[4].y = abs(fft(p))
        figw.data[5].y = abs(fft(s))

figw

interactive(children=(IntSlider(value=200, description='fs', max=600, min=-200), Output()), _dom_classes=('wid…

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'c634666b-5cdb-4d95-b7ac-4103ed569f36',
 …